In [1]:
dataset <- "ARPAL"
state <- "Liguria"


In [2]:
setwd(fs::path_abs("~/Local_Workspace/TesiMag"))
library(arrow, warn.conflicts = FALSE)
library(dplyr, warn.conflicts = FALSE)

source("src/database/tools.R")
source("src/database/insert.R")
source("src/database/write.R")
source("src/database/startup.R")
source("src/database/plot.R")
source("src/database/query/spatial.R")
source("src/database/query/data.R")
source("src/merging/analysis.R")
source("src/merging/display.R")
source("src/merging/combining.R")

conns <- load_dbs()
state_boundaries <- query_boundary(conns$stations, state, "state")


Linking to GEOS 3.11.0, GDAL 3.5.3, PROJ 9.1.0; sf_use_s2() is TRUE



In [3]:
ds_meta <- query_checkpoint_meta(dataset, "qc1", conns$data) |>
    collect()
scia_meta <- query_checkpoint_meta("SCIA", "qc1", conns$data) |>
    collect() |>
    st_md_to_sf() |>
    st_filter(state_boundaries |> st_buffer(dist = 1000), .predicate = st_within) |>
    st_drop_geometry() |>
    filter(!state %in% c("Piemonte", "Emilia-Romagna", "Toscana"), !(network %in% c("Sinottica", "Mareografica")))

metadata <- bind_rows(ds_meta, scia_meta) |> mutate(key = row_number()) #|> mutate(dataset = if_else(dataset == "SCIA", 2L, 1L))
metadata_db <- copy_to(conns$data, metas, "metas_tmp", overwrite = TRUE)
matches <- close_matches(metas, 10000, conns$stations)

data <- query_checkpoint_data(c(dataset, "SCIA"), "qc1", conns$data) |>
    filter(valid) |>
    select(-starts_with("qc_"), -valid) |>
    left_join(metas |> select(dataset, sensor_key, key), by = c("dataset", "sensor_key"))

smatches <- series_matches(data, matches, metas)


In [4]:
analysis1 <- series_matches_analysis(smatches, data, metas)
write_xlsx_analysis(analysis1, "notebooks/integrazioni_regionali/liguria/analysis.xlsx", starts_with("user_code"), starts_with("sensor_id"))


In [5]:
source("notebooks/integrazioni_regionali/liguria/procedures/match_taggers.R")
source("src/merging/tagging.R")
tagged_analysis <- tag_same_series(analysis1) |>
    tag_manual() |>
    tag_mergeable() |>
    default_logic(require_all = TRUE)
tagged_analysis |> write_xlsx_analysis("notebooks/integrazioni_regionali/liguria/same_series_analysis.xlsx", starts_with("user_code"), starts_with("sensor_id"), starts_with("tag_"))


In [6]:
gs <- series_groups(tagged_analysis, metadata, data, tag_same_series)
ranked_series_groups <- rank_series_groups(gs$table, metadata, merging_datasets, desc(dataset), desc(sensor_last))
incomp <- incompatible_merges(data, ranked_series_groups, tagged_analysis, 1L, 0.5)


ERROR: Error in eval(expr, envir, enclos): oggetto 'metadata' non trovato


In [ ]:
incomp |>
    left_join(metadata |> select(key, name, dataset, sensor_key), by = c("key_x" = "key")) |>
    left_join(metadata |> select(key, name, dataset, sensor_key), by = c("key_y" = "key")) |>
    arrange(desc(abs(k0)))


In [ ]:
merge_results <- dynamic_merge(data, ranked_series_groups, metadata, tagged_analysis, 1)
db <- merged_checkpoint(merge_results, metadata, "FVG")
rm(merge_results)
gc()

In [ ]:
db <- query_checkpoint("FVG", "merged", conns$data, all_stations = FALSE)
pd <- ymonthly_availabilities(db$data)
pd$plot + facet_grid(variable ~ .)

In [ ]:
pd <- spatial_availabilities(pd[[2]], db$metadata, state_boundaries)
pd$plot + facet_grid(variable ~ .)